# SLIC Tutorial
- In this hands-on, you implement SLIC, which is one of the methods of unsupervised segmentation.
    - SLIC: Simple Linear Iterative Clustering ([paper](https://infoscience.epfl.ch/record/177415/files/Superpixel_PAMI2011-2.pdf))
<div align="left">
<img src="figure/slic.JPG", width=300, style="margin-left:0">
</div>

    
- You implement the following three parts of SLIC step by step:
    - Task 1: Iinitialize the segments
    - Task 2: Assign each pixel to a segment
    - Task 3: Update the segments

In [ ]:
%matplotlib inline
import numpy as np
import skimage.data
from skimage.util import img_as_float, regular_grid
from skimage.color import rgb2lab
from skimage.segmentation import mark_boundaries
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

## Load image
- You can load your favorite image from skimage.data.
- Color images (Recommended):
    - astronaut
    - chelsea
    - coffee
- Monochrome images:
    - camera
    - coins

In [ ]:
raw_img = skimage.data.chelsea()
plt.imshow(raw_img)

- Convert RGB to [Lab color space](https://en.wikipedia.org/wiki/Lab_color_space) if you chose color images.
    - To make the distance in the color space close to that of human perception.

In [ ]:
image = img_as_float(raw_img)  # Regularize the image into the range of 0-1
if image.ndim == 2:  # Monochrome
    image = image[..., np.newaxis] - 0.5
else:  # Color
    image = rgb2lab(raw_img)  # Convert to Lab color space
    
print(image.shape)

## SLIC algorithm
- Quite similar to k-means clustering. Assign each pixel to a segment (=cluster in k-means).
- The point is the definition of distance: the (weighted) sum of spatial distance and color distance.

### Hyperparameters
- n_segments: the number of segments. It corresponds to K in k-means clustering.
- compactness: decides which of spatial distance and color distance we enphasize. The larger the compactness value is, the more we give weights to the spatial distance; therefore, each segment becomes more square.
- max_iter: 10 is OK.

In [ ]:
n_segments = 100
compactness = 0.1  # Color->10, monochrome->0.1 will be good
max_iter = 3  # Set to 10 for the best result

## Task 1: Initialize the segments
- Create an array "segments" which contains the center coordinates and the mean color values of each segment.

- Requirements:
    - 2-D array formed like:<br>[[y, x, color1, color2, color3],<br>[y, x, color1, color2, color3],<br>...]<br>
    - Initialize x- and y-coordinates as in the following figure.
        - $step = \sqrt{\frac{height \, \times \, width}{n\_segments}}$
        - The final number of segments will sometimes be not equal to n_segments you defined above.
    - Initialize all color values with 0.
    - The order of the segments does not matter.
<div align="left">
<img src="figure/init.png", width=300, style="margin-left:0">
</div>

In [ ]:
height, width = image.shape[:2]
step = int(np.sqrt(height * width) * 1.0 / n_segments)

# WRITE ME!


### Check

In [ ]:
print(segments.shape)

plt.imshow(raw_img)
plt.plot(segments[:, 1], segments[:, 0], 'o', color='r')

## Task 2: Assign each pixel to a segment / Task 3: Update the segments
- Implement the main two steps of SLIC.


- Task 2: Assign each pixel to a segment 
    - Assign each pixel to the <u>**closest**</u> segment.
$$
\begin{align}
d &= \sqrt{(d_{spatial} * compactness)^2 + d_{color}^2}, \\
{\rm where:} \\
d_{spatial} &= \frac{\sqrt{(x-x_c)^2 + (y-y_c)^2}}{step} \\
d_{color} &= \sqrt{(l-l_c)^2 + (a-a_c)^2 + (b-b_c)^2} \\
\end{align}
$$
    - Save the results on "nearest_segments."
    - [Completed] search only the area close to the center of each segment.
    - [Completed] break if no pixel changed its segment.
    
    
- Task 3: Update the segments
    - Compute the average of coordinates and color values of all pixels assigned to each segment, and write them to "segments."

In [ ]:
nearest_segments = np.empty((height, width), dtype=np.intp)
n_segments = len(segments)  # Redefine

In [ ]:
for i in tqdm(range(max_iter)):
    
    # Task 2: Assign each pixel to a segment
    distance = np.ones((height, width)) * 1e8  # Initialize with very large values
    is_changed = False

    for k in range(n_segments):
        
        # Center coordinates of the segment
        cy = segments[k, 0]
        cx = segments[k, 1]

        # Search only the area close to the center of each segment
        y_min = int(max(cy - 2 * step, 0))
        y_max = int(min(cy + 2 * step + 1, height))
        x_min = int(max(cx - 2 * step, 0))
        x_max = int(min(cx + 2 * step + 1, width))

        # Compute the distance and assigne to a segment
        for y in range(y_min, y_max):
            for x in range(x_min, x_max):
                dist = # WRITE ME!

                if dist < distance[y, x]:
                    # WRITE ME!
                    
                    is_changed = True

    # Break if no pixel changed its segment
    if not is_changed:
        break

    # Task 3: Update the segments
    # WRITE ME!
        

### Show the results

In [ ]:
plt.imshow(mark_boundaries(raw_img, nearest_segments))

- Removing small disconnected regions make the results much better.
    - Use a function of skimage.

In [ ]:
from skimage.segmentation._slic import _enforce_label_connectivity_cython

segment_size = height * width * 1.0 / n_segments
min_size = int(0.5 * segment_size)
max_size = int(3 * segment_size)

improved_nearest_segments = _enforce_label_connectivity_cython(nearest_segments[np.newaxis, ...],
                                                               n_segments,
                                                               min_size,
                                                               max_size)[0]

plt.imshow(mark_boundaries(raw_img, improved_nearest_segments))

### Reference
- Implementation by skimage: 
    - [skimage.segmentation.slic](https://github.com/scikit-image/scikit-image/blob/master/skimage/segmentation/slic_superpixels.py)
    - [_slic.pyx](https://github.com/scikit-image/scikit-image/blob/master/skimage/segmentation/_slic.pyx)
- Original paper: [SLIC Superpixels Compared to State-of-the-art Superpixel Methods](https://infoscience.epfl.ch/record/177415/files/Superpixel_PAMI2011-2.pdf)